In [1]:
!pip install mlflow

     ---------------------------------------- 19.7/19.7 MB 9.9 MB/s eta 0:00:00
     -------------------------------------- 233.4/233.4 kB 7.2 MB/s eta 0:00:00
     ------------------------------------- 207.3/207.3 kB 12.3 MB/s eta 0:00:00
     ---------------------------------------- 41.2/41.2 kB ? eta 0:00:00
     ---------------------------------------- 56.7/56.7 kB ? eta 0:00:00
     ------------------------------------- 413.4/413.4 kB 13.0 MB/s eta 0:00:00
     -------------------------------------- 147.6/147.6 kB 8.6 MB/s eta 0:00:00
     ---------------------------------------- 24.8/24.8 MB 9.3 MB/s eta 0:00:00
     ---------------------------------------- 128.2/128.2 kB ? eta 0:00:00
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
     ---------------------------------------- 78.7/78.7 kB 4.3 MB/s eta 0:00:00
  Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
  Using cached aniso8601-9.0.1-py2.py3-none-any.whl (52 kB)
     -------------------------------

In [2]:
import mlflow
# Crear experimento
mlflow.create_experiment("Experimento")
# Tag new experiment
mlflow.set_experiment_tag("churn-modeling","trees")
# Set the experiment
mlflow.set_experiment("Experimento")


<Experiment: artifact_location='file:///c:/Users/Paul/Documents/freelance/docencia/cunef/otros/example-mlflow/01_demos/02_mlflow/mlflow-example/notebooks/mlruns/258821500882448194', creation_time=1712179938498, experiment_id='258821500882448194', last_update_time=1712179938498, lifecycle_stage='active', name='Experimento', tags={}>

In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import mlflow



# Cargar el dataset de Iris
iris = load_iris()
X, y = iris.data, iris.target

# Para simplificar, vamos a convertirlo en un problema binario solo con dos clases
# Eliminaremos la clase '2' del dataset
is_binary_class = y != 2
X_binary, y_binary = X[is_binary_class, :], y[is_binary_class]

# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_binary, y_binary, test_size=0.2, random_state=42)



In [8]:



mlflow.set_experiment("LR Experiment")
# Start a run
mlflow.start_run()
# mlflow.end_run()
# Crear el modelo de regresión logística
model = LogisticRegression(solver='liblinear',n_jobs=1 )  # Usamos liblinear para datasets pequeños

# Entrenar el modelo con los datos
model.fit(X_train, y_train)

# Predecir el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

score = model.score(X, y)
# Log a metric
mlflow.log_metric("score", score)
# Log a parameter
mlflow.log_param("n_jobs",1)
# Log an artifact
#mlflow.log_artifact("")

1

In [10]:
mlflow.sklearn.autolog()

# Train the model
lr = LinearRegression()
lr.fit(X, y)
# Get params
params = lr.get_params(deep=True)
params
mlflow.end_run()    


2024/04/04 14:22:39 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 7432842203d44f939078f115173862ac. Failed operations: [MlflowException("Changing param values is not allowed. Param with key=\'n_jobs\' was already logged with value=\'1\' for run ID=\'7432842203d44f939078f115173862ac\'. Attempted logging new value \'None\'.")]')]


{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}

In [12]:

# Save a model to the local filesystem
#mlflow.sklearn.save_model(model)
# Log a model as an artifact to MLflow Tracking.
#mlflow.sklearn.log_model(model)
#mlflow.log_params(params)

# Load a model from local filesystem or from MLflow Tracking.
#mlflow.sklearn.load_model(model_uri)




In [16]:
def train_test_split_data():
    # Cargar el dataset de Iris
    iris = load_iris()
    X, y = iris.data, iris.target

    # Para simplificar, vamos a convertirlo en un problema binario solo con dos clases
    # Eliminaremos la clase '2' del dataset
    is_binary_class = y != 2
    X_binary, y_binary = X[is_binary_class, :], y[is_binary_class]

    # Dividir el dataset en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_binary, y_binary, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test 

def train_model(train_x,train_y):
    lr = LogisticRegression(solver='liblinear',n_jobs=1 )
    lr.fit(train_x, train_y)
    return lr

def eval_metrics(y_test, y_pred):
    # Evaluar el modelo
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    return accuracy, conf_matrix, class_report



def mlflow_model( MODELS_PATH= "."): 
    mlflow.set_experiment("LR Experiment")

    mlflow.set_tracking_uri("file:{0}/mlruns".format(MODELS_PATH))   
    with mlflow.start_run():
        X_train, X_test, y_train, y_test= train_test_split_data()
        lr = train_model(X_train,y_train)
        predicted_qualities = lr.predict(X_test)
        (accuracy, conf_matrix, class_report) = eval_metrics(y_test, predicted_qualities)
        score = lr.score(X_test, y_test)
        params = lr.get_params(deep=True)



        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("score", score)

        mlflow.log_params(params)

        mlflow.sklearn.log_model(lr, "model")


In [17]:
mlflow_model()